# Indexing and Selecting

Variable, DataArrays, and Datasets in Scipp can be "sliced" in several ways.
The general way is [positional indexing](#Positional-indexing) using indices as in NumPy. 
A second approach is to use [label-based indexing](#Label-based-indexing) which uses actual coordinate values for selection.
Positional and label-based indexing returns *view* into the indexed object and can be used to modify an object in-place.

In addition, [advanced indexing](#Advanced-indexing), which comprises [integer array indexing](#Integer-array-indexing) and [boolean variable indexing](#Boolean-variable-indexing), can be used for more complex selections.
Unlike the aforementioned basic positional and label-based indexing, indexing with integer arrays or boolean variables returns a *copy* of the indexed object.

## Positional indexing

### Overview

Data in a [variable](../generated/classes/scipp.Variable.rst#scipp.Variable), [data array](../generated/classes/scipp.DataArray.rst#scipp.DataArray), or [dataset](../generated/classes/scipp.Dataset.rst#scipp.Dataset) can be indexed in a similar manner to NumPy and xarray.
The dimension to be sliced is specified using a dimension label.
In contrast to NumPy, positional dimension lookup is not available, unless the object being sliced is one-dimensional.
Positional indexing with an integer or an integer range is made via `__getitem__` and `__setitem__` with a dimension label as first argument.
This is available for variables, data arrays, and datasets.
In all cases a *view* is returned, i.e., just like when slicing a [numpy.ndarray](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray) no copy is performed.

### Variables

Consider the following variable:

In [ ]:
import numpy as np
import scipp as sc

var = sc.array(
    dims=['z', 'y', 'x'],
    values=np.random.rand(2, 3, 4),
    variances=np.random.rand(2, 3, 4),
)
sc.show(var)

As when slicing a `numpy.ndarray`, the dimension `'x'` is removed since no range is specified:

In [ ]:
s = var['x', 1]
sc.show(s)
print(s.dims, s.shape)

When a range is specified, the dimension is kept, even if it has extent 1:

In [ ]:
s = var['x', 1:3]
sc.show(s)
print(s.dims, s.shape)

s = var['x', 1:2]
sc.show(s)
print(s.dims, s.shape)

Slicing can be chained arbitrarily:

In [ ]:
s = var['x', 1:4]['y', 2]['x', 1]
sc.show(s)
print(s.dims, s.shape)

The `copy()` method turns a view obtained from a slice into an independent object:`

In [ ]:
s = var['x', 1:2].copy()
s += 1000
var

To avoid subtle and hard-to-spot bugs, positional indexing without dimension label is in general not supported:

In [ ]:
try:
    var[1]
except sc.DimensionError as e:
    print(e)

Scipp makes an exception from this rule in the unambiguous case of 1-D objects:

In [ ]:
var1d = sc.linspace(dim='x', start=0.1, stop=0.2, num=5)
var1d[1]

In [ ]:
var1d[2:4]

Positional index also supports an optional stride (step):

In [ ]:
var['x', 1:4:2]

Negative step sizes are current not supported.

### Data arrays

Slicing for data arrays works in the same way, but some additional rules apply.
Consider:

In [ ]:
a = sc.DataArray(
    data=sc.array(dims=['y', 'x'], values=np.random.rand(2, 3)),
    coords={
        'x': sc.array(dims=['x'], values=np.arange(3.0), unit='m'),
        'y': sc.array(dims=['y'], values=np.arange(2.0), unit='m'),
    },
    masks={'mask': sc.array(dims=['x'], values=[True, False, False])},
)
sc.show(a)
a

As when slicing a variable, the sliced dimension is removed when slicing without range, and kept when slicing with range.

When slicing a data array the following additional rule applies:

- Meta data (coords, masks) that *do not depend on the slice dimension* are marked as *readonly*
- Slicing **without range**:
  - The *coordinates* for the sliced dimension become unaligned.
- Slicing **with a range**:
  - The *coordinates* for the sliced dimension keep their alignment.

The rationale behind this mechanism is as follows.
Meta data is often of a lower dimensionality than data, such as in this example where coords and masks are 1-D whereas data is 2-D.
Elements of meta data entries are thus shared by many data elements, and we must be careful to not apply operations to subsets of data while unintentionally modifying meta data for other unrelated data elements:

In [ ]:
a['x', 0:1].coords['x'] *= 2  # ok, modifies only coord value "private" to this x-slice
try:
    # not ok, would modify coord value "shared" by all x-slices
    a['x', 0:1].coords['y'] *= 2
except sc.VariableError as e:
    print(
        f'\'y\' is shared with other \'x\'-slices and should not be modified by the slice, so we get an error:\n{e}'
    )

In practice, a much more dangerous issue this mechanism protects from is unintentional changes to masks.
Consider

In [ ]:
val = a['x', 1]['y', 1].copy()
val

If we now assign this scalar `val` to a slice at `y=0`, using `=` we need to update the mask.
However, the mask in this example depends only on `x` so it also applies to the slices `y=1`.
If we would allow updating the mask, the following would *unmask data for all* `y`:

In [ ]:
try:
    a['y', 0] = val
except sc.DimensionError as e:
    print(e)

Since we cannot update the mask in a consistent manner the entire operation fails.
Data is not modified.
The same mechanism is applied for binary arithmetic operations such as `+=` where the masks would be updated using a logical OR operation.

The purpose for making coords unaligned when slicing *without* a range is to support useful operations such as:

In [ ]:
a - a['x', 1]  # compute difference compared to data at x=1

If the `x` coord of `a['x', 0]` were aligned, this would fail due to a coord mismatch.
If coord checking is required, use a range-slice such as `a['x', 1:2]`. Compare the two cases shown in the following and make sure to inspect the `dims` and `shape` of all variables (data and coordinates) of the resulting slices (note the tooltip shown when moving the mouse over the name also contains this information):

In [ ]:
sc.show(a['y', 1:2])  # Range of length 1
a['y', 1:2]

In [ ]:
sc.show(a['y', 1])  # No range
a['y', 1]

### Datasets

Slicing for datasets works just like for data arrays.
In addition to making certain coords unaligned and marking certain meta data entries as read-only, slicing a dataset also marks lower-dimensional *data entries* readonly.
Consider a dataset `d`:

In [ ]:
d = sc.Dataset(
    data={
        'a': sc.array(dims=['y', 'x'], values=np.random.rand(2, 3)),
        'b': sc.array(dims=['x', 'y'], values=np.random.rand(3, 2)),
    },
    coords={
        'x': sc.array(dims=['x'], values=np.arange(3.0), unit='m'),
        'y': sc.array(dims=['y'], values=np.arange(2.0), unit='m'),
    },
)
sc.show(d)

and a slice of `d`:

In [ ]:
sc.show(d['y', 0])

Slicing a data item of a dataset should not bring any surprises.
Essentially this behaves like slicing a data array:

In [ ]:
sc.show(d['a']['x', 1:2])

Slicing and item access can be done in arbitrary order with identical results:

In [ ]:
assert sc.identical(d['x', 1:2]['a'], d['a']['x', 1:2])
assert sc.identical(d['x', 1:2]['a'].coords['x'], d.coords['x']['x', 1:2])

## Label-based indexing

### Overview

Data in a [dataset](../generated/classes/scipp.Dataset.rst#scipp.Dataset) or [data array](../generated/classes/scipp.DataArray.rst#scipp.DataArray) can be selected by the coordinate value.
This is similar to pandas [pandas.DataFrame.loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html).
Scipp leverages its ubiquitous support for physical units to provide label-based indexing in an intuitive manner, using the same syntax as [positional indexing](#Positional-indexing).
For example:

- `array['x', 0:3]` selects positionally, i.e., returns the first three element along `'x'`.
- `array['x', 1.2*sc.Unit('m'):1.3*sc.Unit('m')]` selects by label, i.e., returns the elements along `'x'` falling between `1.2 m` and `1.3 m`.

That is, label-based indexing is made via `__getitem__` and `__setitem__` with a dimension label as first argument and a scalar [variable](../generated/classes/scipp.Variable.rst#scipp.Variable) or a Python `slice()` as created by the colon operator `:` from two scalar variables.
In all cases a *view* is returned, i.e., just like when slicing a [numpy.ndarray](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray) no copy is performed.

Consider:

In [ ]:
da = sc.DataArray(
    data=sc.array(dims=['year', 'x'], values=np.random.random((3, 7))),
    coords={
        'x': sc.array(dims=['x'], values=np.linspace(0.1, 0.9, num=7), unit='m'),
        'year': sc.array(dims=['year'], values=[2020, 2023, 2027]),
    },
)
sc.show(da)
da

We can select a slice of `da` based on the `'year'` labels:

In [ ]:
year = sc.scalar(2023)
da['year', year]

In this case `2023` is the second element of the coordinate so this is equivalent to positionally slicing `data['year', 1]` and [the usual rules](#Positional-indexing) regarding dropping dimensions and making dimension coordinates unaligned apply:

In [ ]:
assert sc.identical(da['year', year], da['year', 1])

<div class="alert alert-warning">
    
**Warning**

It is **essential** to not mix up integers and scalar Scipp variables containing an integer.
As in above example, positional indexing yields different slices than label-based indexing.
    
</div>

<div class="alert alert-info">

**Note**

Here, we created `year` using `sc.scalar`.
Alternatively, we could use `year = 2023 * sc.units.dimensionless` which is useful for dimensionful coordinates like `'x'` in this case, see below.
    
</div>

For floating-point-valued coordinates selecting a single point would require an exact match, which is typically not feasible in practice.
Scipp does *not* do fuzzy matching in this case, instead an `IndexError` is raised:

In [ ]:
x = 0.23 * sc.Unit('m')  # Equivalent of sc.scalar(0.23, unit='m')
try:
    # No x coordinate value at this point.
    da['x', x]
except IndexError as e:
    print(str(e))

For such coordinates we may thus use an *interval* to select a *range* of values using the `:` operator:

In [ ]:
x_left = 0.1 * sc.Unit('m')
x_right = 0.4 * sc.Unit('m')
da['x', x_left:x_right]

The selection includes the bounds on the "left" but excludes the bounds on the "right", i.e., we select the half-open interval $x \in [x_{\text{left}},x_{\text{right}})$, closed on the left and open on the right.

The half-open interval implies that we can select consecutive intervals without including any data point in both intervals:

In [ ]:
x_mid = 0.2 * sc.Unit('m')
sc.to_html(da['x', x_left:x_mid])
sc.to_html(da['x', x_mid:x_right])

Just like when slicing positionally one of the bounds can be omitted, to include either everything from the start, or everything until the end:

In [ ]:
da['x', :x_right]

Coordinates used for label-based indexing must be monotonically ordered.
While it is natural to think of slicing in terms of ascending coordinates, the slicing mechanism also works for descending coordinates.

### Bin-edge coordinates

Bin-edge coordinates are handled slightly differently from standard coordinates in label-based indexing.
Consider:

In [ ]:
da = sc.DataArray(
    data=sc.array(dims=['x'], values=np.random.random(7)),
    coords={'x': sc.array(dims=['x'], values=np.linspace(1.0, 2.0, num=8), unit='m')},
)
da

Here `'x'` is a bin-edge coordinate, i.e., its length exceeds the array dimensions by one.
Label-based slicing with a single coord value finds and returns the bin that contains the given coord value:

In [ ]:
x = 1.5 * sc.Unit('m')
da['x', x]

If an interval is provided when slicing with a bin-edge coordinate, the range of bins *containing* the values falling into the right-open interval bounds is selected:

In [ ]:
x_left = 1.3 * sc.Unit('m')
x_right = 1.7 * sc.Unit('m')
da['x', x_left:x_right]

### Limitations

Label-based indexing *not* supported for:

- Multi-dimensional coordinates.
- Non-monotonic coordinates.

The first is a fundamental limitation since a slice cannot be defined in such as case.
In the second case it would in general not be possible to return a view.

### See also

<div class="alert alert-info">

**Note**

When working with binned data, the `bins` property supports a mechanism similar to label-based indexing, `da.bins['param', param_value]`, returning a copy of the selected or filtered coordinate values of individual bin entries.
See [Rearranging and filtering binned data](./binned-data/filtering.rst) for details.
    
</div>

## Advanced indexing

### Integer array indexing

Indexing a variable, data array, or dataset with an integer array (instead of an integer or slice) returns a new variable, data array, or dataset including the elements with the elements at the selected positions.

This is identical to [positional indexing](#Positional-indexing) except that multiple "positions" (but not ranges) can be specified at once, and a single object is returned.

Consider:

In [ ]:
import scipp as sc

var = sc.arange('dummy', 12).fold(dim='dummy', sizes={'x': 6, 'y': 2})
var

Indexing with a list `[1,2,5]` extracts the corresponding slices and return a single output object containing the specified slices:

In [ ]:
var['x', [1, 2, 5]]

<div class="alert alert-info">

**Note**

By necessity &mdash; since the returned elements are generally non-contiguous &mdash; indexing with an integer array returns a *copy* of the input data.
This is in contrast to [positional indexing](#Positional-indexing) and [label-based indexing](#Label-based-indexing) which return a view.

Note that indexing with an index array *always* returns a copy, even if the selected elements form a contiguous range.
    
</div>

For 1-D objects the dimension label can be omitted:

In [ ]:
var1d = sc.linspace(dim='x', start=0.1, stop=0.2, num=5)
var1d[[3, 1, 3]]

### Boolean variable indexing

Indexing a variable, data array, or dataset with a condition variable of `dtype=bool` returns a new variable, data array, or dataset including the elements where the condition is `True`.

The condition variable must be 1-D and must be compatible with the shape of the indexed object.
Consider a 2-D variable:

In [ ]:
import scipp as sc

var = sc.arange('dummy', 12).fold(dim='dummy', sizes={'x': 6, 'y': 2})
var

Indexing with a boolean variable corresponds to extracting rows or columns:

In [ ]:
condition = sc.array(dims=['x'], values=[True, False, False, True, False, False])
var[condition]

In [ ]:
condition = sc.array(dims=['y'], values=[False, True])
var[condition]

<div class="alert alert-info">

**Note**

By necessity &mdash; since the returned elements are generally non-contiguous &mdash; indexing with a condition variable returns a *copy* of the input data.
This is in contrast to [positional indexing](#Positional-indexing) and [label-based indexing](#Label-based-indexing) which return a view.

Note that indexing with a condition variable *always* returns a copy, even if the selected elements form a contiguous range.
    
</div>

Given a multi-dimensional condition variable such as

In [ ]:
condition = var < 5
condition

an indexing attempt will raise an error:

In [ ]:
var[condition]

Unlike [NumPy's boolean array indexing](https://numpy.org/doc/stable/user/basics.indexing.html#boolean-array-indexing) Scipp does not support this, since it would require automatic flattening for the output, which is incompatible with Scipp's philosophy of enforcing labeled dimensions.
If such indexing is required, flatten by hand instead:

In [ ]:
var.flatten(to='elem')[condition.flatten(to='elem')]

When the index object has a bin-edge coordinate along the dimension being index, this coordinate is dropped from the output, since edges from non-adjacent bins are generally not compatible, i.e., we cannot define a sensible output coordinate:

In [ ]:
da = sc.DataArray(var)
da.coords['x'] = sc.arange('x', 7)
da.coords['x2'] = sc.arange('x', 6)
da

In [ ]:
condition = sc.array(dims=['x'], values=[True, False, False, True, False, False])
da[condition]